### 정리된 코드

In [1]:
# ================================
# 라이브러리 설치 (노트북 환경 가정)
# ================================
!pip install dart_fss

# ================================
# 임포트
# ================================
import os                      # 환경변수에서 API 키를 읽기 위해 사용
from pathlib import Path
import math
import re
from typing import List, Optional, Dict

import numpy as np
import pandas as pd
import dart_fss as dart        # DART 비공식 Python 라이브러리
import dart_fss



Error occurred during getting browser(s): random, but was suppressed with fallback.


In [ ]:
# ================================
# 1) API 키 설정
# ================================
API_KEY = os.getenv("DART_API_KEY", "66ce66618f4850247aa36d3d0bea34737980af17").strip()  # 환경변수에서 키를 읽음(없으면 기본 문자열)
if not API_KEY:                                   # 키가 비어 있으면,
    API_KEY = "인증키 입력"                       # 직접 문자열로 넣는 방법(임시)
dart.set_api_key(api_key=API_KEY)                 # dart-fss에 키를 등록해야 API 호출 가능

'66ce66618f4850247aa36d3d0bea34737980af17'

In [250]:

# ================================

# 2) 회사 찾기
# ================================
corp_list = dart.get_corp_list()                  # 전체 회사 목록(오프라인 캐시됨)
cn = input("찾으시는 회사명을 입력하세요")
# corp = corp_list.find_by_stock_code("005930")   # 종목코드로 삼성전자 찾기(예: 005930)
corp = corp_list.find_by_corp_name(cn, exactly=True)[0]  # 회사명으로 찾는 방법

# ================================
# 3) 재무제표 추출(extract_fs)
#   - bgn_de: 시작일(YYYYMMDD). 이 날짜 이후 공시들을 기준으로 재무제표 구성
#   - report_tp: 'annual'(연간), 'half'(반기), 'quarter'(분기)
#   - separate: False=연결, True=개별
# ================================
fs = corp.extract_fs(
    bgn_de="20220101",       # 2018년 1월 1일부터
    report_tp="annual",      # 연간 보고서(사업보고서)
    separate=False,          # 연결 재무제표(일반적으로 이걸 많이 씁니다.)
    progressbar=False
)
# fs.save(filename="연결_연간.xlsx")

# ================================
# 4) 유틸 함수
# ================================
# 4-1) 안전한 나눗셈(0/NaN 방지)
def safe_div(a, b):
    """a/b 계산 시 0 나누기, NaN 발생을 방지하고 NaN을 깔끔히 반환"""
    if a is None or b is None:
        return float('nan')
    if pd.isna(a) or pd.isna(b) or b == 0:
        return float('nan')
    return a / b

# 4-2) 특정 계정값을 읽어오는 도우미 (라벨 열 이름을 유연하게)
#    - df_flat: flatten_statement() 결과
#    - keywords: 포함 검색 키워드
#    - col: 'current', 'prev', 'prev2' ...
#    - label_col: 라벨 열 이름 (기본 후보: label, 계정, 항목, account)
def pick_value(df_flat, keywords, col='current', label_col=None):
    if df_flat is None or len(df_flat) == 0:
        return float('nan')  # 표 자체가 없으면 NaN

    # 라벨 열 자동 추정
    if label_col is None:
        candidates = ['label', '계정', '항목', 'account', 'name']
        label_col = next((c for c in candidates if c in df_flat.columns), None)

    if label_col is None:
        # 라벨 열을 못 찾으면 첫 번째 열을 사용 (마지막 방어)
        label_col = df_flat.columns[0]

    # 불리언 마스크(시리즈) 준비
    mask = pd.Series(False, index=df_flat.index)

    # 키워드 포함 검색 (대소문자 무시, NaN 안전)
    for kw in keywords:
        mask = mask | df_flat[label_col].astype(str).str.contains(kw, case=False, na=False)

    hits = df_flat.loc[mask]
    if hits.empty or col not in df_flat.columns:
        return float('nan')  # 못 찾거나 해당 값 열이 없으면 NaN

    # 가장 위(첫 행)의 지정 열 값을 숫자로 변환 후 반환
    val = pd.to_numeric(hits.iloc[0][col], errors='coerce')
    return float(val) if not pd.isna(val) else float('nan')


# ================================
# 5) 재무제표 가져오기
# ================================
cis_flat = fs['cis']   # 포괄손익계산서
is_flat  = fs['is']    # 포괄손익계산서
bs_flat  = fs['bs']    # 재무상태표
cf_flat  = fs['cf']    # 현금흐름표

from pandas.api.types import is_numeric_dtype

# ================================
# 6) 전처리
# ================================

#손익계산서(is)
# 손익계산서 (is_flat)
if "is_flat" in globals() and isinstance(is_flat, pd.DataFrame) and not is_flat.empty:
    is_flat = is_flat.T
    is_flat.columns = is_flat.columns.get_level_values(-1)
    is_flat.columns = is_flat.iloc[1]
    is_flat = is_flat.reset_index(level=1, drop=True)
    is_flat.index = is_flat.index.str.split('-').str[-1]  # 인덱스 이름 통일
    cond3 = is_flat.index.to_series().astype(str).str.isdigit()
    is_flat = is_flat.loc[cond3]
    is_flat.index = is_flat.index.astype(str).astype(int)



#재무상태표(bs)
bs_flat = bs_flat.T
bs_flat.columns = bs_flat.columns.get_level_values(-1)
bs_flat.columns = bs_flat.iloc[1]
bs_flat = bs_flat.reset_index(level=1, drop=True)
cond = bs_flat.index.to_series().astype(str).str.isdigit()
bs_flat = bs_flat.loc[cond]
bs_flat.index = bs_flat.index.astype(str).astype(int)


#포괄손익계산서(cis)
cis_flat = cis_flat.T
cis_flat.columns = cis_flat.columns.get_level_values(-1)
cis_flat.columns = cis_flat.iloc[1]
cis_flat = cis_flat.drop(cis_flat.index[0:7])
cis_flat = cis_flat.reset_index(level=1, drop=True)
cis_flat.index = cis_flat.index.str.split('-').str[-1]  # 인덱스 이름 통일



#현금흐름표(cf)
cf_flat = cf_flat.T
cf_flat.columns = cf_flat.columns.get_level_values(-1)
cf_flat.columns = cf_flat.iloc[1]
cf_flat = cf_flat.reset_index(level=1, drop=True)
cf_flat.index = cf_flat.index.str.split('-').str[-1]  # 인덱스 이름 통일
cond2 = cf_flat.index.to_series().astype(str).str.isdigit()
cf_flat = cf_flat.loc[cond2]
cf_flat.index = cf_flat.index.astype(str).astype(int)




# ================================
# 1) 별칭 사전 (원본 유지!)
# ================================
# — 당신이 주신 KOR_KEY_ALIASES / SAMSUNG_EXTRA_PASSTHROUGH 그대로 사용한다고 가정 —
# (이 스니펫은 해당 객체들이 이미 상단에 정의되어 있다고 가정합니다.)

# ================================
# 2) 인덱스/이름 정규화 유틸 (원본 유지!)
# ================================
# — 당신이 주신 _normalize_name, _normalize_date_index_like, _unify_df_index_yyyymmdd 등 그대로 사용 —

# ================================
# 3) 컬럼 탐색/숫자 변환 유틸 (원본 유지!)
# ================================
# — 당신이 주신 _pick_first_exist, _to_numeric_safe_series, _resolve_numeric_series, _get_colname, _get_series,
#   _sum_all_aliases, _ensure_series, _safe_div, _prev_by_year_series 등 그대로 사용 —

# ================================
# 3-1) 추가 유틸: 평균(전년도와 산술평균), 부분합 탐색(별칭 사전은 '건드리지 않고' 개별 리스트로 탐색)
# ================================

def _avg_with_prev_year(s: Optional[pd.Series]) -> Optional[pd.Series]:
    if not isinstance(s, pd.Series) or s.empty:
        return None
    prev = _prev_by_year_series(s)
    if not isinstance(prev, pd.Series):
        return None
    out = (s.astype(float) + prev.astype(float)) / 2.0
    return out


def _sum_any(df: Optional[pd.DataFrame], names: list[str]) -> Optional[pd.Series]:
    """KOR_KEY_ALIASES는 건드리지 않고, 임의의 한국어 명칭 리스트로 합산 탐색"""
    return _sum_all_aliases(df, names) if isinstance(df, pd.DataFrame) and not df.empty else None


# ================================
# 3-2) 재무 항목 탐색 (KOR_KEY_ALIASES 외 보강)
# ================================
# * alias 딕셔너리는 변경하지 않고, 여기서만 필요한 항목들을 느슨한 매칭으로 찾아 합산합니다.

# 재고/매출채권
_INVENTORY_ALIASES = [
    "재고자산", "상품", "제품", "원재료", "재공품"
]
_AR_ALIASES = [
    "매출채권", "받을어음및매출채권", "받을어음 및 매출채권", "단기매출채권", "매출채권및기타채권",
    # 금융업(은행/보험) 대체 표기
    "대출채권", "상각후원가 측정 대출채권", "대출금"
]

# 차입금(이자부 부채 계열)
_BORROWINGS_ALIASES = [
    "단기차입금", "유동성장기차입금", "유동성장기부채", "유동성리스부채", "사채", "전환사채", "교환사채",
    "장기차입금", "리스부채", "기타차입금"
]

# 매출원가/감가상각/EBITDA/현금흐름/투자활동 중 유형자산 취득(설비투자)
_COGS_ALIASES = ["매출원가"]
_DA_ALIASES = ["감가상각비", "무형자산상각비", "상각비"]
_EBITDA_ALIASES = ["EBITDA", "상각전영업이익"]

# ================================
# 3-2a) 금융업 프락시 별칭 및 감지
# ================================
_BANK_CA_PROXY_ALIASES = [
    # 유동성 높은 자산들(대략)
    "현금및현금성자산", "현금 및 현금성자산", "단기금융상품",
    "당기손익-공정가치 측정 금융자산", "기타포괄손익-공정가치 측정 금융자산",
    "파생금융자산", "당기법인세자산", "기타유동자산", "기타자산(유동)",
]

_BANK_CL_PROXY_ALIASES = [
    # 단기성/유동성 부채들(대략)
    "예수부채", "당기법인세부채", "기타부채", "단기차입금",
    "유동성장기부채", "유동성리스부채", "리스부채(유동)", "파생금융부채",
]

_FIN_MARKERS = [
    "대출채권", "보험계약부채", "재보험계약부채", "투자계약부채", "예수부채", "보험계약자산",
]

def _is_financial_institution(bs: Optional[pd.DataFrame], cf: Optional[pd.DataFrame]) -> bool:
    def _norms(df: Optional[pd.DataFrame]):
        if not isinstance(df, pd.DataFrame) or df.empty:
            return set()
        return { _normalize_name(c) for c in df.columns }
    cols = _norms(bs) | _norms(cf)
    for m in _FIN_MARKERS:
        if _normalize_name(m) in cols:
            return True
    return False
_CFO_ALIASES = ["영업활동현금흐름", "영업활동으로인한현금흐름", "영업활동으로 인한 현금흐름"]
_CAPEX_ALIASES = [
    "유형자산의취득", "유형자산 취득", "유형자산취득", "설비투자", "건설중인자산의취득"
]

# 당기순이익(지배/비지배 귀속 항목이 여러 가지일 수 있어 느슨하게 탐색)
_NET_INCOME_ALIASES = [
    "당기순이익", "분기순이익", "연결당기순이익",
    "지배기업소유주지분에귀속되는당기순이익", "지배기업 소유주지분에 귀속되는 당기순이익",
]


# ================================
# 4) 메인 계산 함수 (19개 지표 산출)
# ================================

# ================================
# 3-2a) CF 별칭 사전 및 헬퍼 (기존 alias 미변경)
# ================================
CF_KEY_ALIASES = {
    # 영업활동현금흐름(순액)
    "cfo": [
        "영업활동현금흐름", "영업활동으로인한현금흐름", "영업활동으로 인한 현금흐름",
        "영업현금흐름", "영업활동현금흐름(순액)", "영업활동으로부터의현금흐름(순액)",
        "영업활동현금흐름순액", "영업활동으로부터의현금흐름"
    ],
    # 감가상각비(무형 상각 포함) — 여러 항목 합산
    "da": [
        "감가상각비", "무형자산상각비", "상각비", "감가상각및무형자산상각",
        "감가상각비(+)", "무형자산상각비(+)"
    ],
    # 유형자산의 취득 (투자활동 현금유출) — 단일 항목 선택
    "capex": [
        "유형자산의취득", "유형자산 취득", "유형자산취득", "설비투자",
        "유형자산의 취득", "건설중인자산의취득", "기계장치의취득"
    ],
}

def _get_cf_colname(df: Optional[pd.DataFrame], key: str) -> Optional[str]:
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    return _pick_first_exist(df.columns.tolist(), CF_KEY_ALIASES.get(key, []))


def _get_cf_series(df: Optional[pd.DataFrame], key: str) -> Optional[pd.Series]:
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    name = _get_cf_colname(df, key)
    if name is None:
        return None
    return _resolve_numeric_series(df, name)


def _sum_cf_aliases(df: Optional[pd.DataFrame], key: str) -> Optional[pd.Series]:
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    aliases = CF_KEY_ALIASES.get(key, [])
    return _sum_all_aliases(df, aliases)


def compute_metrics_df_flat_kor(
    bs_flat_df: Optional[pd.DataFrame],
    is_flat_df: Optional[pd.DataFrame] = None,
    cis_flat_df: Optional[pd.DataFrame] = None,
    cf_flat_df: Optional[pd.DataFrame] = None,  # ✅ CF 추가
    key_cols: Optional[list[str]] = None,
) -> pd.DataFrame:
    """
    입력: BS / IS / CIS / CF (플랫 테이블, 인덱스=기간)
    출력: 요청하신 19개 핵심 지표만 포함한 DataFrame (index=yyyymmdd 문자열, 내림차순)
    """

    # (A) 기본 방어
    if not isinstance(bs_flat_df, pd.DataFrame) or bs_flat_df.empty:
        cols = [
            "debt_ratio","equity_ratio","debt_dependency_ratio",
            "current_ratio","quick_ratio","interest_coverage_ratio","ebitda_to_total_debt","cfo_to_total_debt","free_cash_flow",
            "operating_margin","roa","roe","net_profit_margin",
            "total_asset_turnover","accounts_receivable_turnover","inventory_turnover",
            "sales_growth_rate","operating_income_growth_rate","total_asset_growth_rate",
        ]
        return pd.DataFrame(columns=cols)

    # 인덱스 통일(yyyymmdd 문자열)
    bs = _unify_df_index_yyyymmdd(bs_flat_df.copy())
    is_pl_df  = _unify_df_index_yyyymmdd(is_flat_df.copy())  if isinstance(is_flat_df,  pd.DataFrame) and not is_flat_df.empty  else None
    cis_pl_df = _unify_df_index_yyyymmdd(cis_flat_df.copy()) if isinstance(cis_flat_df, pd.DataFrame) and not cis_flat_df.empty else None
    cf_df     = _unify_df_index_yyyymmdd(cf_flat_df.copy())  if isinstance(cf_flat_df,  pd.DataFrame) and not cf_flat_df.empty  else None

    # (B) BS 핵심 항목
    ca  = _get_series(bs, "current_assets")          # 유동자산
    cl  = _get_series(bs, "current_liabilities")     # 유동부채
    ncl = _get_series(bs, "noncurrent_liabilities")  # 비유동부채
    tl  = _get_series(bs, "total_liabilities")       # 부채총계
    eqt = _get_series(bs, "equity_total")            # 자본총계
    eqp = _get_series(bs, "equity_parent")           # 지배기업 소유주지분
    eqn = _get_series(bs, "equity_nci")              # 비지배지분
    ta  = _get_series(bs, "total_assets")            # 자산총계

    # 총부채 보정(없으면 유동+비유동)
    if tl is None and (cl is not None or ncl is not None):
        basis_idx = getattr(cl, "index", getattr(ncl, "index", bs.index))
        tl = pd.Series(0.0, index=basis_idx, dtype=float)
        if cl is not None:  tl = tl.add(cl,  fill_value=0.0)
        if ncl is not None: tl = tl.add(ncl, fill_value=0.0)

    # 자본총계 계산 우선순위
    if eqt is not None:
        eq = eqt
    elif (eqp is not None) and (eqn is not None):
        eq = eqp.add(eqn, fill_value=0.0)
    elif (ta is not None) and (tl is not None):
        eq = ta.sub(tl, fill_value=np.nan)
    else:
        eq = None

    # (C) PL 소스: IS 우선, 없으면 CIS
    def _extract_pl(df):
        if not isinstance(df, pd.DataFrame) or df.empty:
            return (None, None, None, None, None, None, df)
        r   = _get_series(df, "revenue")
        rp  = _get_series(df, "revenue_prev")
        oi  = _get_series(df, "operating_income")
        oi0 = _get_series(df, "operating_income_preLLP")
        clo = _get_series(df, "credit_loss")
        fc  = _get_series(df, "finance_costs")
        return (r, rp, oi, oi0, clo, fc, df)

    r1, rp1, oi1, oi01, clo1, fc1, is_pl  = _extract_pl(is_pl_df)
    r2, rp2, oi2, oi02, clo2, fc2, cis_pl = _extract_pl(cis_pl_df)

    # 우선순위(항목별 보충)
    r   = r1   if r1  is not None else r2
    rp  = rp1  if rp1 is not None else rp2
    oi  = oi1  if oi1 is not None else oi2
    oi0 = oi01 if oi01 is not None else oi02
    clo = clo1 if clo1 is not None else clo2
    fc  = fc1  if fc1 is not None else fc2

    # 대표 PL df
    pl_any = is_pl if (isinstance(is_pl, pd.DataFrame) and not is_pl.empty) else cis_pl

    # 수익(근사): 없으면 이자/수수료/보험수익 합산
    if r is None and isinstance(pl_any, pd.DataFrame):
        interest = _sum_all_aliases(pl_any, KOR_KEY_ALIASES["interest_income"])  # 기존 유틸 재사용
        fee      = _sum_all_aliases(pl_any, KOR_KEY_ALIASES["fee_income"])       
        ins      = _sum_all_aliases(pl_any, KOR_KEY_ALIASES["insurance_revenue"]) 
        parts = [p for p in [interest, fee, ins] if isinstance(p, pd.Series)]
        if parts:
            total = parts[0]
            for p in parts[1:]:
                total = total.add(p, fill_value=0.0)
            r = total

    # 영업이익 보충: '신용손실충당금 반영전 영업이익' - '신용손실충당금 전입액'
    if oi is None and oi0 is not None:
        oi = oi0 if clo is None else oi0.sub(clo, fill_value=0.0)

    # 전년도 매출 Fallback
    if (rp is None) or (not isinstance(rp, pd.Series)) or rp.isna().all():
        rp = _prev_by_year_series(r)

    # (D) CF/기타 항목 탐색
    inv  = _sum_any(bs, _INVENTORY_ALIASES)      # 재고자산
    ar   = _sum_any(bs, _AR_ALIASES)             # 매출채권
    debt = _sum_any(bs, _BORROWINGS_ALIASES)     # 총차입금(느슨히 합산)

    cogs = _sum_any(pl_any, _COGS_ALIASES)       # 매출원가
    da_pl = _sum_any(pl_any, _DA_ALIASES)        # D&A (PL에서)
    ebitda_direct = _sum_any(pl_any, _EBITDA_ALIASES)  # EBITDA 직접 표기

    # CF에서 항목 (별칭 사전 기반 — 단일/합산 구분)
    cfo = _get_cf_series(cf_df, "cfo") if isinstance(cf_df, pd.DataFrame) else None
    da_cf = _sum_cf_aliases(cf_df, "da") if isinstance(cf_df, pd.DataFrame) else None
    capex_raw = _get_cf_series(cf_df, "capex") if isinstance(cf_df, pd.DataFrame) else None

    # --------------------
    # ⚙️ 금융업 프락시 적용 (KB금융 등)
    # --------------------
    is_fin = _is_financial_institution(bs, cf_df)
    if is_fin:
        if (ca is None) or (isinstance(ca, pd.Series) and ca.isna().all()):
            ca = _sum_any(bs, _BANK_CA_PROXY_ALIASES)
        if (cl is None) or (isinstance(cl, pd.Series) and cl.isna().all()):
            cl = _sum_any(bs, _BANK_CL_PROXY_ALIASES)
        if (ar is None) or (isinstance(ar, pd.Series) and ar.isna().all()):
            ar = _sum_any(bs, _AR_ALIASES)
        # 은행은 재고자산이 사실상 없음 → 0 처리하여 quick_ratio = current_ratio 되도록
        if (inv is None) or (isinstance(inv, pd.Series) and inv.isna().all()):
            basis_idx = getattr(ca, "index", bs.index)
            inv = pd.Series(0.0, index=basis_idx, dtype=float)

    # EBITDA: 직접 항목 우선, 없으면 OI + (D&A)
    da = da_pl if isinstance(da_pl, pd.Series) else da_cf
    if ebitda_direct is not None:
        ebitda = ebitda_direct
    elif (oi is not None) and (da is not None):
        ebitda = oi.add(da, fill_value=0.0)
    else:
        ebitda = None

    # CapEx: 투자활동에서 "유형자산 취득" 계정 (보통 음수). FCF = CFO - CapEx(절대 유출액)
    if isinstance(capex_raw, pd.Series):
        capex_outflow = capex_raw.copy().astype(float)
        capex_outflow = capex_outflow.where(capex_outflow < 0.0, 0.0).abs()  # 유출만 +값으로
    else:
        capex_outflow = None

    # (E) 공통 인덱스(문자열) 구성
    union_idx = pd.Index(bs.index.astype(str), dtype="object")
    add_list = [r, rp, oi, fc, ca, cl, ta, tl, eq, inv, ar, debt, cogs, ebitda, cfo, capex_outflow]
    for s in add_list:
        if isinstance(s, pd.Series):
            union_idx = union_idx.union(pd.Index(s.index.astype(str), dtype="object"))

    def _on_idx(s):
        return _ensure_series(s, union_idx)

    # 인덱스 정렬 적용
    ca = _on_idx(ca); cl = _on_idx(cl); ta = _on_idx(ta); tl = _on_idx(tl); eq = _on_idx(eq)
    r  = _on_idx(r);  rp = _on_idx(rp); oi = _on_idx(oi);  fc = _on_idx(fc)
    inv = _on_idx(inv); ar = _on_idx(ar); debt = _on_idx(debt); cogs = _on_idx(cogs)
    ebitda = _on_idx(ebitda); cfo = _on_idx(cfo); capex_outflow = _on_idx(capex_outflow)

    # eq 보정
    if isinstance(eq, pd.Series) and eq.isna().all() and (ta is not None) and (tl is not None):
        eq = ta - tl

    # (F) 평균치(자산/자본/AR/재고)
    avg_assets  = _on_idx(_avg_with_prev_year(ta))
    avg_equity  = _on_idx(_avg_with_prev_year(eq))
    avg_ar      = _on_idx(_avg_with_prev_year(ar))
    avg_inv     = _on_idx(_avg_with_prev_year(inv))

    # (G) 보조 항목
    quick_assets = None
    if (ca is not None) and (inv is not None):
        quick_assets = ca.sub(inv, fill_value=np.nan)

    # (H) 19개 지표 계산
    debt_ratio                 = _safe_div(tl, eq, union_idx)                # 부채비율
    equity_ratio               = _safe_div(eq, ta, union_idx)                # 자기자본비율
    debt_dependency_ratio      = _safe_div(debt, ta, union_idx)              # 차입금의존도(총차입금/총자산)

    current_ratio              = _safe_div(ca, cl, union_idx)                # 유동비율
    quick_ratio                = _safe_div(quick_assets, cl, union_idx)      # 당좌비율
    interest_coverage_ratio    = _safe_div(oi, fc, union_idx)                # 이자보상배율
    # 기본 계산
    ebitda_to_total_debt = _safe_div(ebitda, tl, union_idx)

    # ⚠️ Fallback 1: 연말(YYYY1231) 키로 재정렬하여 보완
    def _to_year_end_series(s: Optional[pd.Series]) -> Optional[pd.Series]:
        if not isinstance(s, pd.Series) or s.empty:
            return None
        idx = pd.Index([
            (YEAR_RE.search(str(i)).group(0) + "1231") if YEAR_RE.search(str(i)) else str(i)
            for i in s.index.astype(str)
        ], dtype="object")
        s2 = s.copy()
        s2.index = idx
        return s2[~s2.index.duplicated(keep="last")]

    try:
        e_y = _to_year_end_series(ebitda)
        tl_y = _to_year_end_series(tl)
        if isinstance(e_y, pd.Series) and isinstance(tl_y, pd.Series):
            ye_idx = pd.Index(sorted(set(e_y.index).union(set(tl_y.index))), dtype="object")
            ye_ratio = _safe_div(_ensure_series(e_y, ye_idx), _ensure_series(tl_y, ye_idx), ye_idx)
            ebitda_to_total_debt = _ensure_series(ebitda_to_total_debt, union_idx).combine_first(ye_ratio.reindex(union_idx))
    except Exception:
        pass

    # ⚠️ Fallback 2: 연월(YYYYMM) → 말일(YYYYMMDD)로 보정하여 보완
    def _to_ym_end_series(s: Optional[pd.Series]) -> Optional[pd.Series]:
        if not isinstance(s, pd.Series) or s.empty:
            return None
        import calendar
        new_idx = []
        for i in s.index.astype(str):
            digits = re.sub(r"\D", "", str(i))
            if len(digits) >= 6:
                y = int(digits[:4]); m = int(digits[4:6])
                try:
                    last_day = calendar.monthrange(y, m)[1]
                    new_idx.append(f"{y:04d}{m:02d}{last_day:02d}")
                except Exception:
                    new_idx.append(str(i))
            else:
                new_idx.append(str(i))
        s2 = s.copy(); s2.index = pd.Index(new_idx, dtype="object")
        return s2[~s2.index.duplicated(keep="last")]

    try:
        e_m = _to_ym_end_series(ebitda)
        tl_m = _to_ym_end_series(tl)
        if isinstance(e_m, pd.Series) and isinstance(tl_m, pd.Series):
            ym_idx = pd.Index(sorted(set(e_m.index).union(set(tl_m.index))), dtype="object")
            ym_ratio = _safe_div(_ensure_series(e_m, ym_idx), _ensure_series(tl_m, ym_idx), ym_idx)
            ebitda_to_total_debt = _ensure_series(ebitda_to_total_debt, union_idx).combine_first(ym_ratio.reindex(union_idx))
    except Exception:
        pass

    cfo_to_total_debt          = _safe_div(cfo, tl, union_idx)               # CFO/총부채 = _safe_div(cfo, tl, union_idx)               # CFO/총부채
    free_cash_flow             = None
    if (cfo is not None) and (capex_outflow is not None):
        free_cash_flow = cfo.sub(capex_outflow, fill_value=0.0)

    operating_margin           = _safe_div(oi, r, union_idx)                 # 영업이익률

    # 당기순이익 탐색 (PL 우선)
    net_income = _on_idx(_sum_any(pl_any, _NET_INCOME_ALIASES))

    roa = _safe_div(net_income, avg_assets, union_idx)                       # ROA(평균총자산 기준)
    roe = _safe_div(net_income, avg_equity, union_idx)                       # ROE(평균자기자본 기준)
    net_profit_margin          = _safe_div(net_income, r, union_idx)         # 순이익률

    total_asset_turnover       = _safe_div(r, avg_assets, union_idx)         # 총자산회전율
    accounts_receivable_turnover = _safe_div(r, avg_ar, union_idx)           # 매출채권회전율
    inventory_turnover         = _safe_div(cogs if isinstance(cogs, pd.Series) else r, avg_inv, union_idx)
    # 금융업은 재고 개념이 없어 구조적으로 계산 불가 → NaN 유지(표시는 N/A 처리)
    try:
        if is_fin:
            inventory_turnover = pd.Series(np.nan, index=union_idx, dtype=float)
    except NameError:
        pass  # 재고회전율

    sales_growth_rate          = _safe_div(r - _on_idx(_prev_by_year_series(r)),  _on_idx(_prev_by_year_series(r)),  union_idx)
    operating_income_growth_rate = _safe_div(oi - _on_idx(_prev_by_year_series(oi)), _on_idx(_prev_by_year_series(oi)), union_idx)
    total_asset_growth_rate    = _safe_div(ta - _on_idx(_prev_by_year_series(ta)), _on_idx(_prev_by_year_series(ta)), union_idx)

    # (I) 결과 테이블(19개 지표만)
    out = pd.DataFrame(index=union_idx)
    out.index.name = "date"

    out["debt_ratio"] = debt_ratio
    out["equity_ratio"] = equity_ratio
    out["debt_dependency_ratio"] = debt_dependency_ratio

    out["current_ratio"] = current_ratio
    out["quick_ratio"] = quick_ratio
    out["interest_coverage_ratio"] = interest_coverage_ratio
    out["ebitda_to_total_debt"] = ebitda_to_total_debt
    out["cfo_to_total_debt"] = cfo_to_total_debt
    out["free_cash_flow"] = _ensure_series(free_cash_flow, union_idx)

    out["operating_margin"] = operating_margin
    out["roa"] = roa
    out["roe"] = roe
    out["net_profit_margin"] = net_profit_margin

    out["total_asset_turnover"] = total_asset_turnover
    out["accounts_receivable_turnover"] = accounts_receivable_turnover
    out["inventory_turnover"] = inventory_turnover

    out["sales_growth_rate"] = sales_growth_rate
    out["operating_income_growth_rate"] = operating_income_growth_rate
    out["total_asset_growth_rate"] = total_asset_growth_rate

    # 날짜 내림차순 정렬
    try:
        out = out.sort_index(ascending=False, key=lambda s: pd.to_datetime(s, format="%Y%m%d", errors="coerce"))
    except Exception:
        out = out.sort_index(ascending=False)

    return out


# ================================
# 5) 예시 실행부 (환경에 변수 있을 때만)
# ================================
if "bs_flat" in globals():
    result_df = compute_metrics_df_flat_kor(
        bs_flat_df=bs_flat,
        is_flat_df=(is_flat if "is_flat" in globals() else None),
        cis_flat_df=(cis_flat if "cis_flat" in globals() else None),
        cf_flat_df=(cf_flat if "cf_flat" in globals() else None),
        key_cols=None,
    )

    # 보고용 19개 지표
    cols_order = [
        "debt_ratio","equity_ratio","debt_dependency_ratio",
        "current_ratio","quick_ratio","interest_coverage_ratio","ebitda_to_total_debt","cfo_to_total_debt","free_cash_flow",
        "operating_margin","roa","roe","net_profit_margin",
        "total_asset_turnover","accounts_receivable_turnover","inventory_turnover",
        "sales_growth_rate","operating_income_growth_rate","total_asset_growth_rate",
    ]
    to_show = result_df.loc[:, [c for c in cols_order if c in result_df.columns]].copy()

    # 포맷팅: %/배
    def format_table(df: pd.DataFrame) -> pd.DataFrame:
        df2 = df.copy()
        pct_cols = [
            "debt_ratio","equity_ratio","debt_dependency_ratio",
            "current_ratio","quick_ratio",
            "operating_margin","roa","roe","net_profit_margin",
            "sales_growth_rate","operating_income_growth_rate","total_asset_growth_rate",
        ]
        times_cols = [
            "interest_coverage_ratio","ebitda_to_total_debt","cfo_to_total_debt",
            "total_asset_turnover","accounts_receivable_turnover","inventory_turnover",
        ]
        for c in pct_cols:
            if c in df2.columns:
                df2[c] = (df2[c] * 100).round(2).astype(str) + "%"
        for c in times_cols:
            if c in df2.columns:
                df2[c] = df2[c].round(2)
        if "free_cash_flow" in df2.columns:
            df2["free_cash_flow"] = df2["free_cash_flow"].round(0)
                # 금융업의 재고회전율은 비적용 → 표시만 N/A로
        if "inventory_turnover" in df2.columns:
            df2["inventory_turnover"] = df2["inventory_turnover"].where(df2["inventory_turnover"].notna(), "N/A")
        return df2

    # 최신 기간 1건 보기
    tmp = to_show.copy()
    tmp["_period"] = pd.to_datetime(tmp.index.astype(str), format="%Y%m%d", errors="coerce")
    tmp = tmp.sort_values("_period").drop(columns="_period")
    to_show_latest = tmp.tail(1)

    print(format_table(to_show_latest))


RuntimeError: ('Could not find an annual report', 'Unexpected Error')

### 반복문

In [258]:
# 준비물 가정:
# - from dart_fss import dart  (또는 당신 환경의 dart 객체)
# - pandas as pd, numpy as np
# - 위에서 정의한: safe_div, pick_value, compute_metrics_df_flat_kor 등 전처리/유틸 함수들

# =========================================================
# 0) 기업명 리스트 예시
# =========================================================
corp_names = [
    "롯데쇼핑",
    "대한항공",
    "현대자동차",
    "에코프로",
    "한솔테크닉스"
    # ... 여기 원하는 기업명을 계속 추가
]

# =========================================================
# 1) 개별 기업 1개를 처리하는 함수
#    - 최신 연도 1개 행(19개 지표)을 반환 (pd.Series)
#    - 실패 시 None 반환
# =========================================================
def run_pipeline_for_corp(corp_name: str,
                        bgn_de="20220101",
                        report_tp="annual",
                        separate=False) -> pd.Series | None:
    try:
        corp_list = dart.get_corp_list()
        matches = corp_list.find_by_corp_name(corp_name, exactly=True)
        if not matches:
            # 정확 일치 실패: 느슨한 검색 보조
            matches = corp_list.find_by_corp_name(corp_name, exactly=False)
        if not matches:
            print(f"[WARN] 회사명을 찾을 수 없음: {corp_name}")
            return None

        corp = matches[0]

        # 재무제표 추출
        fs = corp.extract_fs(
            bgn_de=bgn_de,
            report_tp=report_tp,
            separate=separate,
            progressbar=False
        )

        # 표 얻기
        cis_flat = fs['cis']   # 포괄손익계산서
        is_flat  = fs['is']    # 손익계산서
        bs_flat  = fs['bs']    # 재무상태표
        cf_flat  = fs['cf']    # 현금흐름표

        # ------------------------
        # 아래 전처리는 당신 코드 그대로 재현
        # ------------------------

        # is_flat
        if isinstance(is_flat, pd.DataFrame) and not is_flat.empty:
            _is = is_flat.T
            _is.columns = _is.columns.get_level_values(-1)
            _is.columns = _is.iloc[1]
            _is = _is.reset_index(level=1, drop=True)
            _is.index = _is.index.str.split('-').str[-1]
            cond3 = _is.index.to_series().astype(str).str.isdigit()
            _is = _is.loc[cond3]
            _is.index = _is.index.astype(str).astype(int)
        else:
            _is = None

        # bs_flat
        if not isinstance(bs_flat, pd.DataFrame) or bs_flat.empty:
            print(f"[WARN] 재무상태표 없음: {corp_name}")
            return None

        _bs = bs_flat.T
        _bs.columns = _bs.columns.get_level_values(-1)
        _bs.columns = _bs.iloc[1]
        _bs = _bs.reset_index(level=1, drop=True)
        cond = _bs.index.to_series().astype(str).str.isdigit()
        _bs = _bs.loc[cond]
        _bs.index = _bs.index.astype(str).astype(int)

        # cis_flat
        if isinstance(cis_flat, pd.DataFrame) and not cis_flat.empty:
            _cis = cis_flat.T
            _cis.columns = _cis.columns.get_level_values(-1)
            _cis.columns = _cis.iloc[1]
            _cis = _cis.drop(_cis.index[0:7])
            _cis = _cis.reset_index(level=1, drop=True)
            _cis.index = _cis.index.str.split('-').str[-1]
        else:
            _cis = None

        # cf_flat
        if isinstance(cf_flat, pd.DataFrame) and not cf_flat.empty:
            _cf = cf_flat.T
            _cf.columns = _cf.columns.get_level_values(-1)
            _cf.columns = _cf.iloc[1]
            _cf = _cf.reset_index(level=1, drop=True)
            _cf.index = _cf.index.str.split('-').str[-1]
            cond2 = _cf.index.to_series().astype(str).str.isdigit()
            _cf = _cf.loc[cond2]
            _cf.index = _cf.index.astype(str).astype(int)
        else:
            _cf = None

        # 지표 계산 (당신 함수)
        result_df = compute_metrics_df_flat_kor(
            bs_flat_df=_bs,
            is_flat_df=_is,
            cis_flat_df=_cis,
            cf_flat_df=_cf,
            key_cols=None,
        )

        # 최신 기간 1건 Series 추출
        if result_df is None or result_df.empty:
            print(f"[WARN] 지표 계산 결과 없음: {corp_name}")
            return None

        # 최신 한 행 (index가 yyyymmdd 문자열, 내림차순 정렬되어 있다고 가정)
        latest_idx = result_df.index[0]
        latest_row = result_df.loc[latest_idx]

        # 보고 포맷(선택): %/배/정수 등은 외부에서 포맷하세요.
        return latest_row

    except Exception as e:
        print(f"[ERROR] {corp_name}: {e}")
        return None


# =========================================================
# 2) 리스트 반복 → 최종 테이블 (index=기업명, columns=19지표)
# =========================================================
def build_metrics_table(corp_names: list[str]) -> pd.DataFrame:
    rows = {}
    for name in corp_names:
        s = run_pipeline_for_corp(name)
        if s is not None:
            rows[name] = s  # pd.Series(19개 지표)
    if not rows:
        return pd.DataFrame()
    out = pd.DataFrame.from_dict(rows, orient="index")
    out.index.name = "corp_name"
    # 원하시면 컬럼 순서 고정
    cols_order = [
        "debt_ratio","equity_ratio","debt_dependency_ratio",
        "current_ratio","quick_ratio","interest_coverage_ratio","ebitda_to_total_debt","cfo_to_total_debt","free_cash_flow",
        "operating_margin","roa","roe","net_profit_margin",
        "total_asset_turnover","accounts_receivable_turnover","inventory_turnover",
        "sales_growth_rate","operating_income_growth_rate","total_asset_growth_rate",
    ]
    out = out[[c for c in cols_order if c in out.columns]]
    return out

# =========================================================
# 3) 실행 & 보기
# =========================================================
metrics_by_corp = build_metrics_table(corp_names)

# 보기 좋은 포맷(선택)
def format_table_for_report(df: pd.DataFrame) -> pd.DataFrame:
    df2 = df.copy()
    pct_cols = [
        "debt_ratio","equity_ratio","debt_dependency_ratio",
        "current_ratio","quick_ratio",
        "operating_margin","roa","roe","net_profit_margin",
        "sales_growth_rate","operating_income_growth_rate","total_asset_growth_rate",
    ]
    times_cols = [
        "interest_coverage_ratio","ebitda_to_total_debt","cfo_to_total_debt",
        "total_asset_turnover","accounts_receivable_turnover","inventory_turnover",
    ]
    for c in pct_cols:
        if c in df2.columns:
            df2[c] = (df2[c] * 100).round(2).astype(str) + "%"
    for c in times_cols:
        if c in df2.columns:
            df2[c] = df2[c].astype(float).round(2)
    if "free_cash_flow" in df2.columns:
        df2["free_cash_flow"] = pd.to_numeric(df2["free_cash_flow"], errors="coerce").round(0)
    if "inventory_turnover" in df2.columns:
        df2["inventory_turnover"] = df2["inventory_turnover"].where(df2["inventory_turnover"].notna(), "N/A")
    return df2

print(format_table_for_report(metrics_by_corp))


          debt_ratio equity_ratio debt_dependency_ratio current_ratio  \
corp_name                                                               
롯데쇼핑         128.98%       43.67%                37.86%        56.09%   
대한항공         328.82%       23.32%                40.98%        68.44%   
현대자동차        182.52%        35.4%                46.43%        145.6%   
에코프로         112.02%       47.17%                  0.1%        121.5%   
한솔테크닉스        71.75%       58.22%                19.77%       123.38%   

          quick_ratio  interest_coverage_ratio  ebitda_to_total_debt  \
corp_name                                                              
롯데쇼핑           40.43%                    -0.51                  0.02   
대한항공           31.81%                     3.54                  0.11   
현대자동차         109.83%                    15.85                   NaN   
에코프로           65.01%                    -1.03                   NaN   
한솔테크닉스         68.66%                     0.62          

Object `corp_list.find_by_corp_list` not found.
